In [5]:
import os
import numpy as np
import pandas as pd


In [40]:
#read in each song from each mood folder, create mel-spectrogram, and save as png in same folder
happy_df = pd.DataFrame()
sad_df = pd.DataFrame()

# Load the audio file
moods = ['happy','sad'] #,'energetic','calm'
for mood in moods:
    pngs = []
    path_of_the_directory= 'mp3s/'+mood
    print("PNGs in "+mood+" folder:")
    for filename in os.listdir(path_of_the_directory):
        f = os.path.join(path_of_the_directory,filename)
        try:
            if (os.path.isfile(f)) & ('.png' in filename):
                pngs.append(filename)
        except Exception:
            pass
    if mood == 'happy':
        happy_df['images'] = pngs
        happy_df['label'] = np.zeros(len(pngs),dtype=int)
    else:
        sad_df['images'] = pngs
        sad_df['label'] = np.ones(len(pngs),dtype=int)

PNGs in happy folder:
PNGs in sad folder:


In [50]:
happy_df
# sad_df

,images,label
0,Like I Can.mp3.png,0
1,She Love the G's.mp3.png,0
2,Still Sippin Mickies (feat. Happy).mp3.png,0
3,Be Like That - feat. Swae Lee & Khalid.mp3.png,0
4,Pure Gold.mp3.png,0
...,...,...
368,Slidin' (feat. Kodak Black).mp3.png,0
369,Inside My Head.mp3.png,0
370,Thelma + Louise.mp3.png,0
371,Free Yourself.mp3.png,0


In [63]:
from sklearn.model_selection import train_test_split
#randomly sample 325 songs from each
df = pd.concat([happy_df.sample(n=325, random_state=1), sad_df.sample(n=325, random_state=1)])
#create train and test split
df_train, df_test = train_test_split(df, test_size=0.23) #0.23 gets exactly 500 of the 650 which is a nice number

In [65]:
df_train
df_test

,images,label
321,Fo Sev Boi (feat. Dopey & Happy).mp3.png,0
364,Ride or Die (feat. Phoebe Ryan).mp3.png,0
159,All Three.mp3.png,1
320,If By Chance.mp3.png,1
186,Just Friends.mp3.png,0
...,...,...
196,You Get Me So High.mp3.png,1
93,Dandelion.mp3.png,0
268,Before.mp3.png,1
367,Softly.mp3.png,0


In [78]:
y = df_train['label']
y

27     0
64     0
11     1
118    1
210    1
      ..
170    1
368    0
38     1
267    1
98     0
Name: label, Length: 500, dtype: int64

In [71]:
from tensorflow.keras.preprocessing import image
from tqdm import tqdm

def horrid_read(df, path):
    img_size = 300
    X = []
    for i in tqdm(range(df.shape[0])):
        img = image.load_img(path[i], target_size=(img_size, img_size, 3))
        img = image.img_to_array(img).astype('float32')
        X.append(img)
    return np.array(X)
for mood in moods:
    try:
        path = 'mp3s/'+ mood + df_train['images']
        X = horrid_read(df_train, path)
    except Exception:
            pass

X.shape

ImportError: generic_type: type "TransactionToken" is already registered!

In [76]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

# Reset Model
tf.keras.backend.clear_session()

ImportError: generic_type: type "TransactionToken" is already registered!

In [73]:
conv_base  = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False)
conv_base.trainable = False

conv_base.summary()

NameError: name 'keras' is not defined

In [ ]:
inputs = keras.Input(shape=(300, 300, 3))
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(4, activation="softmax")(x)
model = keras.Model(inputs, outputs)

model.summary()

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="best_model.keras",
        save_best_only=True,
        monitor="accuracy")
]

history = model.fit(X,y,epochs=100,batch_size=128,verbose=1,callbacks=callbacks,validation_split = 0.15)

In [ ]:
import matplotlib.pyplot as plt
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()